Convert Yolo format dataset to Coco dataset

In [5]:
import os
import sys

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', 'src')))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', 'src/data')))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', 'src/utils')))

In [3]:
YOLO_DATASET_PATH = '../dataset/yolo'
COCO_DATASET_PATH = '../dataset/coco'

DRONE = 'drone'

In [14]:
import json
import os
from PIL import Image
import file_utils as futils

# # Set the paths for the input and output directories
# input_dir = YOLO_DATASET_PATH
# output_dir = COCO_DATASET_PATH

def create_coco_cataset(input_dir, output_dir):

    # Define the categories for the COCO dataset
    categories = [{"id": 0, "name": DRONE}]

    # Define the COCO dataset dictionary
    coco_dataset = {
        "info": {},
        "licenses": [],
        "categories": categories,
        "images": [],
        "annotations": []
    }

    for split in ['test', 'train', 'val']:
        image_dir = os.path.join(input_dir, 'images', split)
        labels_dir = os.path.join(input_dir, 'labels', split)

        # Loop through the images in the input directory
        for image_file in os.listdir(image_dir):
    
            # Load the image and get its dimensions
            image_path = os.path.join(image_dir, image_file)
            image = Image.open(image_path)
            width, height = image.size
    
            id = int(image_file.split('.')[0].split('_')[1])
            # Add the image to the COCO dataset
            image_dict = {
                "id": id,
                "width": width,
                "height": height,
                "file_name": image_path
            }
            coco_dataset["images"].append(image_dict)
    
            # Load the bounding box annotations for the image
            with open(os.path.join(labels_dir, f'{image_file.split(".")[0]}.txt')) as f:
                annotations = f.readlines()
    
            # Loop through the annotations and add them to the COCO dataset
            for ann in annotations:
                x, y, w, h = map(float, ann.strip().split()[1:])
                x_min, y_min = int((x - w / 2) * width), int((y - h / 2) * height)
                x_max, y_max = int((x + w / 2) * width), int((y + h / 2) * height)
                ann_dict = {
                    "id": len(coco_dataset["annotations"]),
                    "image_id": id,
                    "category_id": 0,
                    "bbox": [x_min, y_min, x_max - x_min, y_max - y_min],
                    "area": (x_max - x_min) * (y_max - y_min),
                    "iscrowd": 0
                }
                coco_dataset["annotations"].append(ann_dict)

        # Save the COCO dataset to a JSON file
        with open(os.path.join(output_dir, 'annotations.json'), 'w') as f:
            json.dump(coco_dataset, f)

create_coco_cataset(YOLO_DATASET_PATH, COCO_DATASET_PATH)


In [ ]:
import os
import json
from PIL import Image
from tqdm import tqdm


def yolo_to_coco(image_dir, label_dir, output_dir):
	# Define categories
	categories = [{'id': 0, 'name': 'person'}]

	# Initialize data dict
	data = {'train': [], 'validation': [], 'test': []}

	# Loop over splits
	for split in ['train', 'validation', 'test']:
		split_data = {'info': {}, 'licenses': [], 'images': [], 'annotations': [], 'categories': categories}

		# Get image and label files for current split
		image_files = sorted(os.listdir(image_dir))
		label_files = sorted(os.listdir(label_dir))

		# Loop over images in current split
		cumulative_id = 0
		with tqdm(total=len(image_files), desc=f'Processing {split} images') as pbar:
			for i, filename in enumerate(image_files):
				image_path = os.path.join(image_dir, filename)
				im = Image.open(image_path)
				im_id = i + 1

				split_data['images'].append({
					'id': im_id,
					'file_name': filename,
					'width': im.size[0],
					'height': im.size[1]
				})

				# Get labels for current image
				label_path = os.path.join(label_dir, os.path.splitext(filename)[0] + '.txt')
				with open(label_path, 'r') as f:
					yolo_data = f.readlines()

				for line in yolo_data:
					class_id, x_center, y_center, width, height = line.split()
					class_id = int(class_id)
					bbox_x = (float(x_center) - float(width) / 2) * im.size[0]
					bbox_y = (float(y_center) - float(height) / 2) * im.size[1]
					bbox_width = float(width) * im.size[0]
					bbox_height = float(height) * im.size[1]

					split_data['annotations'].append({
						'id': cumulative_id,
						'image_id': im_id,
						'category_id': class_id,
						'bbox': [bbox_x, bbox_y, bbox_width, bbox_height],
						'area': bbox_width * bbox_height,
						'iscrowd': 0
					})

					cumulative_id += 1

				pbar.update(1)

		data[split] = split_data

	# Save data to JSON files
	for split in ['train', 'validation', 'test']:
		filename = os.path.join(output_dir, f'{split}.json')
		with open(filename, 'w') as f:
			json.dump({'data': data[split]}, f)

	return data



In [ ]:
image_dir = '/home/alberto/Dataset/train/images'
label_dir = '/home/alberto/Dataset/train/labels'
output_dir = './'
coco_data = yolo_to_coco(image_dir, label_dir, output_dir)